# TensorBoard Basic Usage

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

print("tensorflow version:", tf.__version__)

tensorflow version: 2.0.0


## Data

fashion MNIST データを用いる。

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

print(x_train.shape)
print(y_train.shape)

print(x_train.dtype)
print(y_train.dtype)

(60000, 28, 28)
(60000,)
uint8
uint8


In [3]:
x_train = x_train.reshape(-1, 28 * 28) / 255
x_train = x_train.astype(np.float32)

x_test = x_test.reshape(-1, 28 * 28) / 255
x_test = x_test.astype(np.float32)

In [4]:
y_train = y_train[:, np.newaxis]
y_test = y_test[:, np.newaxis]

## Model

In [5]:
def build_model():
    """Return a toy keras sequential model."""
    
    model = keras.models.Sequential([
        keras.layers.Dense(128, activation="relu", input_shape=(28 * 28, )),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dense(10, activation="softmax")
    ])
    
    return model

## Logging with TensorBoard

### Keras Model.fit

In [6]:
model = build_model()
optimizer = keras.optimizers.Adam()

model.compile(
    optimizer,
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [7]:
logdir = os.path.join("logs", "basic", "model_fit")
tb_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model.fit(
    x_train, y_train,
    batch_size = 128,
    epochs=10,
    verbose=1,
    callbacks=[tb_callback],
    validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 5s 82us/sample - loss: 0.5295 - accuracy: 0.8115 - val_loss: 0.4275 - val_accuracy: 0.8472
Epoch 2/10
60000/60000 [==============================] - 3s 47us/sample - loss: 0.3748 - accuracy: 0.8631 - val_loss: 0.3739 - val_accuracy: 0.8651
Epoch 3/10
60000/60000 [==============================] - 3s 48us/sample - loss: 0.3365 - accuracy: 0.8763 - val_loss: 0.3652 - val_accuracy: 0.8673
Epoch 4/10
60000/60000 [==============================] - 3s 45us/sample - loss: 0.3123 - accuracy: 0.8829 - val_loss: 0.3483 - val_accuracy: 0.8712
Epoch 5/10
60000/60000 [==============================] - 3s 50us/sample - loss: 0.2957 - accuracy: 0.8903 - val_loss: 0.3365 - val_accuracy: 0.8771
Epoch 6/10
60000/60000 [==============================] - 3s 46us/sample - loss: 0.2815 - accuracy: 0.8956 - val_loss: 0.3442 - val_accuracy: 0.8732
Epoch 7/10
60000/60000 [==============================] 

### Manually

In [8]:
# make datasets for train and test
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(128)
test_dataset = test_dataset.batch(128)

In [9]:
# loss function
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

# metrics
train_loss = tf.keras.metrics.Mean("train_loss", dtype=tf.float32)
test_loss = tf.keras.metrics.Mean("test_loss", dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy("train_accuracy")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy("test_accuracy")

In [10]:
@tf.function
def train_step(model, optimizer, x_batch, y_batch):
    with tf.GradientTape() as tape:
        preds = model(x_batch, training=True)
        loss = loss_fn(y_batch, preds)
        
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(y_batch, preds)

@tf.function   
def test_step(model, x_batch, y_batch):
    preds = model(x_batch)
    loss = loss_fn(y_batch, preds)
    
    test_loss(loss)
    test_accuracy(y_batch, preds)

In [11]:
train_logdir = os.path.join("logs", "basic", "manually", "train")
test_logdir = os.path.join("logs", "basic", "manually", "validation")

train_summary_writer = tf.summary.create_file_writer(logdir=train_logdir)
test_summary_writer = tf.summary.create_file_writer(logdir=test_logdir)

In [12]:
model = build_model()
optimizer = keras.optimizers.Adam(learning_rate=0.0001)

for epoch in range(10):
    
    # train steps
    for (x_batch, y_batch) in train_dataset:
        train_step(model, optimizer, x_batch, y_batch)
        
    with train_summary_writer.as_default():
        tf.summary.scalar("epoch_loss", train_loss.result(), step=epoch)
        tf.summary.scalar("epoch_accuracy", train_accuracy.result(), step=epoch)
        
    # test steps
    for (x_batch, y_batch) in test_dataset:
        test_step(model, x_batch, y_batch)
        
    with test_summary_writer.as_default():
        tf.summary.scalar("epoch_loss", test_loss.result(), step=epoch)
        tf.summary.scalar("epoch_accuracy", test_accuracy.result(), step=epoch)
        
    # reset metrics
    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()